# Setup

## Imports

In [9]:
from joblib import load
# from sklearn
import pandas as pd

## Variables

In [10]:
RANDOM_STATE = 1432
salaries = load("salaries")

# Modelling

In [11]:
salaries = pd.get_dummies(salaries)
X = salaries.drop(columns="salary_in_usd")
y = salaries["salary_in_usd"]

In [12]:
X

,work_year,experience_level,remote_ratio,company_size,job_title_Data Analyst,job_title_Data Engineer,job_title_Data Scientist,job_title_ML Engineer,job_title_Manager,job_title_Other,company_location_AS,company_location_EU,company_location_NA,company_location_Other
0,2020,0.333333,0.0,1.0,0,0,1,0,0,0,0,1,0,0
2,2020,0.666667,0.5,0.5,0,1,0,0,0,0,0,1,0,0
3,2020,0.333333,0.0,0.0,1,0,0,0,0,0,0,0,1,0
4,2020,0.666667,0.5,1.0,0,0,0,1,0,0,0,0,1,0
5,2020,0.000000,1.0,1.0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,2022,0.666667,1.0,0.5,0,1,0,0,0,0,0,0,1,0
603,2022,0.666667,1.0,0.5,0,1,0,0,0,0,0,0,1,0
604,2022,0.666667,0.0,0.5,1,0,0,0,0,0,0,0,1,0
605,2022,0.666667,1.0,0.5,1,0,0,0,0,0,0,0,1,0


# Evaluation